In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
from glob import glob
#import workspace_utils
#from workspace_utils import active_session
import cv2                
import matplotlib.pyplot as plt                        
from tqdm import tqdm

from PIL import ImageFile
from PIL import Image
ImageFile.LOAD_TRUNCATED_IMAGES = True

from torchvision import datasets
import torchvision.transforms as transforms

import torch
import torchvision.models as models
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import os 
classes_list = os.listdir('../input/cat-breeds-dataset/images')
print (classes_list)

In [ ]:
# # Creating Train / Val / Test folders (One time use)
import os
import numpy as np
import shutil
import random
root_dir = '../input/cat-breeds-dataset/images/' # data root path (data origin)
working_dir = '../working/cat-breeds-dataset/images/'
classes_dir = classes_list #total labels

num_classes = len(classes_list)

val_ratio = 0.15
test_ratio = 0.05



In [ ]:
if (os.path.isdir('../working/') == False):
    #shutil.rmtree('../working/')
    os.listdir('../working/')
    !os.mkdir(working_dir)



In [ ]:
if (os.path.isdir('../working/cat-breeds-dataset') == False):
    for cls in classes_dir:
        os.makedirs(working_dir +'train/' + cls)
        os.makedirs(working_dir +'val/' + cls)
        os.makedirs(working_dir +'test/' + cls)


        # Creating partitions of the data after shuffling
        src = root_dir + cls # Folder to copy images from

        allFileNames = os.listdir(src)
        #np.random.shuffle(allFileNames)
        train_FileNames, val_FileNames, test_FileNames = np.split(np.array(allFileNames),
                                                                  [int(len(allFileNames)* (1 - (val_ratio + test_ratio))), 
                                                                   int(len(allFileNames)* (1 - test_ratio))])


        train_FileNames = [src+ '/' + name for name in train_FileNames.tolist()]
        val_FileNames = [src + '/' +name for name in val_FileNames.tolist()]
        test_FileNames = [src + '/' +name for name in test_FileNames.tolist()]


        # Copy-pasting images
        print(cls)
        for name in train_FileNames:
            shutil.copy(name, working_dir +'train/' + cls)

        for name in val_FileNames:
            shutil.copy(name, working_dir +'val/' + cls)

        for name in test_FileNames:
            shutil.copy(name, working_dir +'test/' + cls)

        print(cls, ' complete.')





In [ ]:
import torchvision.models as models
import torch.nn as nn
import torch

## TODO: Specify model architecture 
# Load the pretrained model from pytorch


model_resnet50 = models.resnet50(pretrained=True)

# Freeze training for all "features" layers
#for param in model_transfer.features.parameters():
for param in model_resnet50.parameters():
    param.requires_grad = False    

model_resnet50.fc = nn.Linear(2048, num_classes, bias=True) 
    
for param in model_resnet50.fc.parameters():
    param.requires_grad = True

# print out the model structure
print(model_resnet50)
    
    


model_vgg16 = models.vgg16(pretrained=True)

for param in model_vgg16.parameters():
    param.requires_grad = False    
    
model_vgg16.classifier[6] = nn.Linear(model_vgg16.classifier[3].out_features, num_classes)
model_vgg16.classifier[6].requires_grad = True

# print out the model structure

print(model_vgg16)


if torch.cuda.is_available():
    model_vgg16 = model_vgg16.cuda()
    model_resnet50 = model_resnet50.cuda()
    use_cuda = True
else:
    use_cuda = False
print(use_cuda)

In [ ]:
## Specify appropriate transforms, and batch_sizes

standard_normalization = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                              std=[0.229, 0.224, 0.225])

data_transform = {'train': transforms.Compose([transforms.RandomResizedCrop(224),
                                     transforms.RandomHorizontalFlip(),
                                     transforms.ToTensor(),
                                     standard_normalization]),
                   'valid': transforms.Compose([transforms.Resize(256),
                                     transforms.CenterCrop(224),
                                     transforms.ToTensor(),
                                     standard_normalization]),
                   'test': transforms.Compose([transforms.Resize(size=(224,224)),
                                     transforms.ToTensor(), 
                                     standard_normalization])
                  }

train_dir = working_dir + 'train'
valid_dir = working_dir + 'val'
test_dir = working_dir + 'test'

train_data = datasets.ImageFolder(train_dir, transform=data_transform['train'])
validation_data = datasets.ImageFolder(valid_dir, transform=data_transform['valid'])
test_data = datasets.ImageFolder(test_dir, transform=data_transform['test'])

# define dataloader parameters
batch_size = 20
num_workers = 0

# prepare data loaders
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, 
                                           num_workers=num_workers, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, 
                                          num_workers=num_workers, shuffle=True)
valid_loader = torch.utils.data.DataLoader(validation_data, batch_size=batch_size, 
                                          num_workers=num_workers, shuffle=True)


loaders = {
    'train': train_loader,
    'valid': valid_loader,
    'test': test_loader
}




In [ ]:

### TODO: select loss function
criterion = nn.CrossEntropyLoss()

### TODO: select optimizer
optimizer = optim.SGD(model_resnet50.parameters(), lr=0.001, momentum=0.5)




In [ ]:
## TODO complete epoch and training batch loops
## These loops should update the classifier-weights of this model
## And track (and print out) the training loss over time

def train(n_epochs, loaders, model, use_cuda, optimizer, criterion, save_path):
    valid_loss_min = np.Inf
    for epoch in range(1, n_epochs+1):
      # keep track of training and validation loss
        train_loss = 0.0
        valid_loss = 0.0
       

        ###################
        # train the model #
        ###################
        model.train()
        #for data, target in loaders_scratch:
        for batch_idx, (data, target) in enumerate(loaders['train']):
            # move tensors to GPU if CUDA is available
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            # clear the gradients of all optimized variables
            optimizer.zero_grad()
            # forward pass: compute predicted outputs by passing inputs to the model
           
            output = model(data)
           
            # calculate the batch loss
            loss = criterion(output, target)
            # backward pass: compute gradient of the loss with respect to model parameters
            loss.backward()
            # perform a single optimization step (parameter update)
            optimizer.step()
            # update training loss
            #train_loss += loss.item()*data.size(0)
            ## record the average training loss, using something like
            train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.data - train_loss))
                
        ######################    
        # validate the model #
        ######################
        model.eval()
        #for data, target in valid_loader:
        for batch_idx, (data, target) in enumerate(loaders['valid']):
            # move tensors to GPU if CUDA is available
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            # forward pass: compute predicted outputs by passing inputs to the model
            output = model(data)
            # calculate the batch loss
            loss = criterion(output, target)
            # update average validation loss 
            valid_loss = valid_loss + ((1 / (batch_idx + 1)) * (loss.data - valid_loss))
           

        # calculate average losses
        train_loss = train_loss/len(train_loader.dataset)
        valid_loss = valid_loss/len(valid_loader.dataset)

        # print training/validation statistics 
        print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
            epoch, train_loss, valid_loss))

        # save model if validation loss has decreased
        if valid_loss <= valid_loss_min:
            print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
            valid_loss_min,
            valid_loss))
            torch.save(model.state_dict(), save_path)
            valid_loss_min = valid_loss
    
    return model

In [ ]:
model = train(30, loaders, model_resnet50, use_cuda, optimizer, 
                      criterion, '../working/model.pt')

In [ ]:
def test(loaders, model, criterion, use_cuda):

    # monitor test loss and accuracy
    test_loss = 0.
    correct = 0.
    total = 0.

    for batch_idx, (data, target) in enumerate(loaders['test']):
        # move to GPU
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the loss
        loss = criterion(output, target)
        # update average test loss 
        test_loss = test_loss + ((1 / (batch_idx + 1)) * (loss.data - test_loss))
        # convert output probabilities to predicted class
        pred = output.data.max(1, keepdim=True)[1]
        # compare predictions to true label
        correct += np.sum(np.squeeze(pred.eq(target.data.view_as(pred))).cpu().numpy())
        total += data.size(0)
            
    print('Test Loss: {:.6f}\n'.format(test_loss))

    print('\nTest Accuracy: %2d%% (%2d/%2d)' % (
        100. * correct / total, correct, total))


In [ ]:
test(loaders, model, criterion, use_cuda)